want to compare the increase of cabi rides between maryland and dc

2022,2023,2024

steps
filter the data set for stations and rides in dc and maryland:
1) take the complete dataset, turn it into geodataframe
2) join it with maryland gdf
3) make plot to compare the difference of growth between them

dive deeper into maryland / prince george county for 2023 -2024

In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
import datetime 
from geopy import distance
import folium
from folium.plugins import MarkerCluster
from folium.features import GeoJsonTooltip
from branca.colormap import LinearColormap
from collections import Counter
import json
from shapely.geometry import Point
import geopandas as gpd
from shapely.geometry import shape

In [2]:
data_types = {
    "rideable_type": "category", 
    "start_station_name": "category", 
    "end_station_name": "category", 
    "member_casual":"category",
    # "ride_id":"uint32",
    "time_of_day":"category",
    "trip_type":"category"}

In [3]:
bikes = pd.read_csv("bikes_clean.csv",parse_dates= ["started_at", "ended_at"], low_memory=False)

In [4]:
cabi_2024 = pd.read_csv("cabi_2024.csv",parse_dates= ["started_at", "ended_at"], low_memory=False)

In [5]:
cabi_updated = pd.concat([bikes, cabi_2024],ignore_index=True,axis=0)

In [ ]:
cabi_updated.shape

(16808320, 20)

In [6]:
# # Create a GeoDataFrame for the start stations
geometry = [Point(xy) for xy in zip(cabi_updated['start_lng'], cabi_updated['start_lat'])]
cabi_updated_gdf = gpd.GeoDataFrame(cabi_updated, geometry=geometry, crs="EPSG:4326")



In [10]:
cabi_updated_gdf =cabi_updated_gdf.drop(columns = ["start_station_id","end_station_id"])

In [12]:
cabi_updated_gdf["rideable_type"] = cabi_updated_gdf["rideable_type"].astype("category")
cabi_updated_gdf["start_station_name"] = cabi_updated_gdf["start_station_name"].astype("category")
cabi_updated_gdf["end_station_name"] = cabi_updated_gdf["end_station_name"].astype("category")
cabi_updated_gdf["member_casual"] = cabi_updated_gdf["member_casual"].astype("category")
cabi_updated_gdf['ride_id'] = pd.Series(range(1, len(bikes) + 1), dtype='uint32')

In [13]:
cabi_updated_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16808320 entries, 0 to 16808319
Data columns (total 19 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   ride_id                float64 
 1   rideable_type          category
 2   started_at             object  
 3   ended_at               object  
 4   start_station_name     category
 5   end_station_name       category
 6   member_casual          category
 7   start_lat              float64 
 8   start_lng              float64 
 9   end_lat                float64 
 10  end_lng                float64 
 11  start_time             object  
 12  end_time               object  
 13  start_hour             float64 
 14  end_hour               float64 
 15  trip_duration_minutes  float64 
 16  time_of_day            object  
 17  Unnamed: 0             float64 
 18  geometry               geometry
dtypes: category(4), float64(9), geometry(1), object(5)
memory usage: 2.0+ GB


In [14]:
cabi_updated_gdf =cabi_updated_gdf.drop(columns = ["ride_id","start_time","end_time","start_hour","end_hour","Unnamed: 0"])

In [ ]:
# cabi_updated_gdf['started_at'] = pd.to_datetime(cabi_updated_gdf['started_at'])#, format='%Y-%m-%d %H:%M:%S')
# cabi_updated_gdf['ended_at'] = pd.to_datetime(cabi_updated_gdf['ended_at'])
# cabi_updated_gdf['year'] = cabi_updated_gdf['started_at'].dt.year
# cabi_updated_gdf['month'] = cabi_updated_gdf['started_at'].dt.month

In [16]:
cabi_updated_gdf.to_csv("cabi_updated_gdf.csv")

# sample

In [ ]:
# # cabi updated gdf sample 

# # 108.000 rows sample (3000 per month)
# samples = []

# # Loop over years and months to sample 1,000 rows for each month
# for year in range(2021, 2025):  # Covering 2021, 2022, and 2024
#     for month in range(1, 13):  # Months from 1 to 12
#         # Filter for the specific year and month
#         filtered_data = cabi_updated_gdf[(cabi_updated_gdf["year"] == year) & (cabi_updated_gdf["month"] == month)]
        
#         # Sample 1,000 rows if there are enough rows available
#         if len(filtered_data) >= 3000:
#             sample = filtered_data.sample(n=3000, random_state=42)
#         else:  # If less than 3,000 rows, take all available rows
#             sample = filtered_data
            
#         samples.append(sample)

# # Combine all the samples into a single DataFrame
# bikes_monthly_sample_3000 = pd.concat(samples, ignore_index=True)

In [ ]:
# bikes_monthly_sample_3000.to_csv("bikes_sample_updated.csv", index=False)

In [17]:
# uploading ward info so I can filter out DC area
with open ("Wards_from_2022.geojson") as i:
    wards = json.loads(i.read())

features =wards["features"]

ward_gdf = gpd.GeoDataFrame(
    pd.DataFrame([feature['properties'] for feature in features]),  # Extract properties as attributes
    geometry=[shape(feature['geometry']) for feature in features],  # Convert geometries
    crs="EPSG:4326" )

In [19]:
ward_gdf = ward_gdf[["WARD","NAME","geometry"]]

In [20]:
dc_wards_gdf = gpd.sjoin(cabi_updated_gdf, ward_gdf, how="left", predicate="within")

In [21]:
with open ("Maryland_Physical_Boundaries_-_County_Boundaries_(Detailed).geojson") as i:
    maryland = json.loads(i.read())

features = maryland["features"]

#GDF
maryland_gdf = gpd.GeoDataFrame(
    pd.DataFrame([feature['properties'] for feature in features]),  # Extract properties as attributes
    geometry=[shape(feature['geometry']) for feature in features],  # Convert geometries
    crs="EPSG:4326")
    

In [27]:
maryland_gdf

,OBJECTID,COUNTY,DISTRICT,COUNTY_FIP,COUNTYNUM,CREATION_D,LAST_UPDAT,Shape_Length,Shape_Area,geometry
0,1,Allegany,6.0,1,1,2010-01-28T00:00:00Z,2010-01-28T00:00:00Z,3.814949,0.115028,"MULTIPOLYGON (((-78.34284 39.72224, -78.34303 ..."
1,2,Anne Arundel,5.0,3,2,2006-04-18T00:00:00Z,2006-04-18T00:00:00Z,10.040214,0.111483,"MULTIPOLYGON (((-76.606 39.22913, -76.60503 39..."
2,3,Baltimore,4.0,5,3,2006-10-09T00:00:00Z,2006-10-09T00:00:00Z,9.470194,0.162506,"MULTIPOLYGON (((-76.56796 39.68592, -76.56795 ..."
3,4,Baltimore City,0.0,510,24,2006-04-18T00:00:00Z,2009-11-16T00:00:00Z,1.978908,0.021987,"MULTIPOLYGON (((-76.52967 39.37196, -76.52975 ..."
4,5,Calvert,5.0,9,4,2010-01-28T00:00:00Z,2010-01-28T00:00:00Z,3.576389,0.057350,"MULTIPOLYGON (((-76.64212 38.76921, -76.64191 ..."
5,6,Caroline,2.0,11,5,2007-05-21T00:00:00Z,2008-07-30T00:00:00Z,3.078015,0.086078,"MULTIPOLYGON (((-75.74764 39.14188, -75.74647 ..."
6,7,Carroll,7.0,13,6,2008-06-16T00:00:00Z,2012-01-17T00:00:00Z,3.179273,0.122033,"MULTIPOLYGON (((-76.86867 39.49261, -76.86904 ..."
7,8,Cecil,2.0,15,7,2006-04-18T00:00:00Z,2008-08-20T00:00:00Z,6.069369,0.094686,"MULTIPOLYGON (((-75.82727 39.69693, -75.82726 ..."
8,9,Charles,5.0,17,8,2009-06-08T00:00:00Z,2009-06-08T00:00:00Z,5.004585,0.123129,"MULTIPOLYGON (((-77.07823 38.69004, -77.07788 ..."
9,10,Dorchester,1.0,19,9,2007-02-08T00:00:00Z,2007-02-22T00:00:00Z,25.068431,0.145041,"MULTIPOLYGON (((-75.81565 38.69825, -75.81515 ..."


In [24]:
dc_wards_gdf = dc_wards_gdf.drop(columns=["index_right"])

In [28]:
maryland_gdf=maryland_gdf[["COUNTY","geometry"]]

In [29]:
# # Perform spatial join with the maryland gdf
dc_maryland_gdf= gpd.sjoin(dc_wards_gdf, maryland_gdf, how="left", predicate="within")

In [ ]:
dc_maryland_gdf[["NAME","COUNTY"]]

,NAME,COUNTY
0,Ward 2,NaN
1,NaN,Prince George's
2,Ward 1,NaN
3,Ward 1,NaN
4,NaN,NaN
...,...,...
143995,Ward 4,NaN
143996,Ward 5,NaN
143997,NaN,NaN
143998,NaN,NaN


In [ ]:
dc_maryland_gdf = dc_maryland_gdf[dc_maryland_gdf["NAME"].notnull()| dc_maryland_gdf["COUNTY"].notnull()]

In [32]:
dc_maryland_gdf['area'] = np.where(dc_maryland_gdf['WARD'].notnull(), 'DC', 'Maryland')

In [ ]:
dc_maryland_gdf.to_csv("dc_maryland_gdf_full_updated.csv")